In [1]:
#!pip install beautifulsoup
#!wget https://dumps.wikimedia.org/enwiki/20190420/enwiki-20190420-pages-articles-multistream1.xml-p10p30302.bz2
#!bunzip2 enwiki-20190420-pages-articles-multistream1.xml-p10p30302.bz2    

In [28]:
# Count lines, words, characters in XML data file
!wc enwiki-20190420-pages-articles-multistream1.xml-p10p30302

 4586335 74563548 642819893 enwiki-20190420-pages-articles-multistream1.xml-p10p30302


In [4]:
from bs4 import BeautifulSoup
import pandas as pd

In [5]:
def get_pages_from_xml_file(filename, start_tag='<page>', end_tag='</page>'):
    """Yields each page from the specified XML data file."""
    page = None
    with open(filename) as f:
        for line in f:
            if start_tag in line:
                page = []
                page.append(line)
            elif end_tag in line:
                page.append(line)
                page_xml = ''.join(page)
                yield page_xml
                page = None
            else:
                if page is not None:
                    page.append(line)

In [10]:
filename = 'enwiki-20190420-pages-articles-multistream1.xml-p10p30302'
pages = list(get_pages_from_xml_file(filename))

In [16]:
def get_title_from_page_xml(page_xml):
    """Returns the title of the given page."""
    
    soup = BeautifulSoup(page_xml, 'lxml')
    return soup.select_one('title').text

In [8]:
len(pages)  # Show the number of pages

19822

In [24]:
%time titles = [get_title_from_page_xml(page) for page in pages]

CPU times: user 25.6 s, sys: 1.13 s, total: 26.7 s
Wall time: 26.9 s


In [31]:
titles[:5] + ['...'] + titles[-5:]  # Show the first and last 5 titles

['AccessibleComputing',
 'Anarchism',
 'AfghanistanHistory',
 'AfghanistanGeography',
 'AfghanistanPeople',
 '...',
 'The Lord of the Rings/One Ring',
 'Tax Freedom Day',
 'Tax',
 'Transhumanism',
 'TARDIS']

In [9]:
print(pages[0])  # Print the first page

  <page>
    <title>AccessibleComputing</title>
    <ns>0</ns>
    <id>10</id>
    <redirect title="Computer accessibility" />
    <revision>
      <id>854851586</id>
      <parentid>834079434</parentid>
      <timestamp>2018-08-14T06:47:24Z</timestamp>
      <contributor>
        <username>Godsy</username>
        <id>23257138</id>
      </contributor>
      <comment>remove from category for seeking instructions on rcats</comment>
      <model>wikitext</model>
      <format>text/x-wiki</format>
      <text xml:space="preserve">#REDIRECT [[Computer accessibility]]

{{R from move}}
{{R from CamelCase}}
{{R unprintworthy}}</text>
      <sha1>42l0cvblwtb4nnupxm6wo000d27t6kf</sha1>
    </revision>
  </page>



In [49]:
def get_text_from_page_xml(page):
    """Returns the text of the given page."""
    soup = BeautifulSoup(page, 'lxml')
    return soup.select_one('text').text


In [50]:
page = pages[0]
soup = BeautifulSoup(page, 'lxml')
text = get_text_from_page_xml(page)
text, type(text)

('#REDIRECT [[Computer accessibility]]\n\n{{R from move}}\n{{R from CamelCase}}\n{{R unprintworthy}}',
 str)

In [51]:
data = []
for page in pages:
    row = {
        'title': get_title_from_page_xml(page),
        'text': get_text_from_page_xml(page),
    }
    data.append(row)

In [52]:
df = pd.DataFrame(data)
df.head()

,text,title
0,#REDIRECT [[Computer accessibility]]\n\n{{R fr...,AccessibleComputing
1,{{redirect2|Anarchist|Anarchists|the fictional...,Anarchism
2,#REDIRECT [[History of Afghanistan]]\n\n{{Redi...,AfghanistanHistory
3,#REDIRECT [[Geography of Afghanistan]]\n\n{{Re...,AfghanistanGeography
4,#REDIRECT [[Demographics of Afghanistan]]\n\n{...,AfghanistanPeople


### Challenge

#### 1. Create a Pandas dataframe containing the title and text of each page.

* Implement the `get_text_from_page_xml` function above.
* Re-create the dataframe with the text field filled in.

#### 2. Identify the five pages that have the _longest_ text.

* Find the length of each page's `<text>...</text>` element and add it to your dataframe.
* Sort the data frame by text length, descending.
* What are the titles of the five longest articles?

In [29]:
df['text_length'] = df['text'].map(lambda x: len(x))

In [31]:
df.head()

,text,title,text_length
0,#REDIRECT [[Computer accessibility]]\n\n{{R fr...,AccessibleComputing,94
1,{{redirect2|Anarchist|Anarchists|the fictional...,Anarchism,102191
2,#REDIRECT [[History of Afghanistan]]\n\n{{Redi...,AfghanistanHistory,90
3,#REDIRECT [[Geography of Afghanistan]]\n\n{{Re...,AfghanistanGeography,92
4,#REDIRECT [[Demographics of Afghanistan]]\n\n{...,AfghanistanPeople,95


In [36]:
df.sort_values(by = ['text_length'], ascending=False).head()

,text,title,text_length
10165,{{short description|Wikipedia list page}}\n{{s...,List of compositions by Johann Sebastian Bach,432964
15129,{{about|the country}}\n{{pp-semi-indef}}\n{{pp...,Pakistan,376957
15287,{{Redirect|Philippine|the town in the Netherla...,Philippines,357950
9299,{{Short description|Overview of relations}}{{U...,Foreign relations of India,317725
8838,{{short description|Aspect of history}}\n{{pp-...,History of India,316816
